In [112]:
# Dependencies and setup
import pandas as pd

# Load data files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student data files and store them each into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [113]:
tot_schools = len(school_data_complete["school_name"].unique())
#print(tot_schools)

tot_students = len(school_data_complete["Student ID"].unique())
#print (tot_students)

school_data["budget"] = school_data["budget"]
tot_budget = school_data["budget"].sum()
#print(tot_budget)

math_avg = round(school_data_complete["math_score"].mean(), 2)
#print(math_avg)

read_avg = round(school_data_complete["reading_score"].mean(), 2)
#print(read_avg)

math_70_data = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
math_70 = round(len(math_70_data)/tot_students * 100, 2)
#print (math_70)

read_70 = round(len(school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]) / tot_students * 100, 2)
#print(read_70)

math_and_read_70 = round(len(school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70), :]) / tot_students * 100, 2)
#print(mathandread_70)

dist_sum_df = pd.DataFrame({"Total Schools": tot_schools,
                         "Total Students": tot_students, 
                          "Total Budget": tot_budget,
                          "Average Math Score": math_avg,
                          "Average Reading Score": read_avg,
                          "% Passing Math": math_70,
                          "% Passing Reading": read_70,
                          "% Overall Passing": math_and_read_70}, index = [0])
dist_sum_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [324]:
# School Summary
gp_school = school_data_complete.groupby(["school_name"])

#Take the max value of each column of interest within the Groupby since the values within the columns are all the same
school_type = gp_school["type"].max()
school_students = gp_school["size"].max()
school_budget = gp_school["budget"].max()

per_student_budget = school_budget/school_students
avg_math = round(gp_school["math_score"].mean(), 2)
avg_read = round(gp_school["reading_score"].mean(), 2)

# Look at original merged data and apply conditional to parse out passing math grades
per_pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
# Group the parsed out data by "school_name" to match School Summary result df. Count number of passing scores.
per_pass_math = per_pass_math.groupby(["school_name"]).count()
# Pull out "math_score" column and divide by previously calculated school_student to prepare to load into School Summary result df.
per_pass_math = round(per_pass_math["math_score"]/school_students * 100, 2)
#per_pass_math

# Look at original merged data and apply conditional to parse out passing reading grades
per_pass_read = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
# Group the parsed out data by "school_name" to match School Summary result df. Count number of passing scores.
per_pass_read = per_pass_read.groupby(["school_name"]).count()
# Pull out "math_score" column and divide by previously calculated school_student to prepare to load into School Summary result df.
per_pass_read = round(per_pass_read["reading_score"]/school_students * 100, 2)
#per_pass_read


per_pass_overall = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70), :]
per_pass_overall = per_pass_overall.groupby(["school_name"]).count()
# Choosing the "size" column here, doesn't matter. Makes more sense then either math or reading columns since it's technically both, but they all hold the same value
per_pass_overall = round(per_pass_overall["size"]/school_students * 100, 2)
#per_pass_overall

#School Summary result df:
school_sum_df = pd.DataFrame({"School Type": school_type,
                              "Total Students": school_students,
                              "Total School Budget": school_budget,
                              "Per Student Budget": per_student_budget,
                              "Average Math Score": avg_math,
                              "Average Reading Score": avg_read,
                              "% Passing Math": per_pass_math,
                              "% Passing Reading": per_pass_read,
                              "% Overall Passing": per_pass_overall, 
                              })
# Rename the index column
school_sum_df.index.names = ["School Name"]
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


In [325]:
# School Summary
#school_group = school_data_complete.groupby(["school_name", "type"])
#g_schoolname_df = school_group["Student ID"].count().to_frame()
#g_schoolname_df["Budget"] = school_group["budget"].max()
#g_schoolname_df["Per Student Budget"] = g_schoolname_df["Budget"]/g_schoolname_df["Student ID"]
#g_schoolname_df["Average Math Score"] = school_group["math_score"].mean()
#g_schoolname_df["Average Reading Score"] = school_group["reading_score"].mean()
#math_pass_df = school_data_complete[["school_name","type","Student ID","math_score"]]
#NOTTHISmath_pass_df["Passed Math"] = math_pass_df.groupby(["school_name", "type"])["math_score"].apply(lambda x: (x >=70).sum())
#g_schoolname_df.reset_index().set_index("school_name")

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [328]:
# To sort from highest to lowest, ascending=False must be passed in
#freedom_df = happiness_df.sort_values("Freedom", ascending=False)
school_sum_df_sorthigh = school_sum_df.sort_values("% Overall Passing", ascending = False)
school_sum_df_sorthigh.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [329]:
school_sum_df_sortlow = school_sum_df.sort_values("% Overall Passing", ascending = True)
school_sum_df_sortlow.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [373]:
# Similar to School Summary section
#per_pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]

avg_math_9th = school_data_complete.loc[(school_data_complete["grade"] == "9th"), :]
avg_math_9th = avg_math_9th.groupby(["school_name"]).mean()["math_score"]
#avg_math_9th

avg_math_10th = school_data_complete.loc[(school_data_complete["grade"] == "10th"), :]
avg_math_10th = avg_math_10th.groupby(["school_name"]).mean()["math_score"]

avg_math_11th = school_data_complete.loc[(school_data_complete["grade"] == "11th"), :]
avg_math_11th = avg_math_11th.groupby(["school_name"]).mean()["math_score"]

avg_math_12th = school_data_complete.loc[(school_data_complete["grade"] == "12th"), :]
avg_math_12th = avg_math_12th.groupby(["school_name"]).mean()["math_score"]

math_by_grade_df = pd.DataFrame({"9th": round(avg_math_9th, 2),
                                 "10th": round(avg_math_10th, 2),
                                 "11th": round(avg_math_11th, 2),
                                 "12th": round(avg_math_12th, 2)
                                })

# Rename the index column
math_by_grade_df.index.names = ["School Name"]

math_by_grade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [374]:
# Same as Math Score by Grade section

avg_read_9th = school_data_complete.loc[(school_data_complete["grade"] == "9th"), :]
avg_read_9th = avg_read_9th.groupby(["school_name"]).mean()["reading_score"]

avg_read_10th = school_data_complete.loc[(school_data_complete["grade"] == "10th"), :]
avg_read_10th = avg_read_10th.groupby(["school_name"]).mean()["reading_score"]

avg_read_11th = school_data_complete.loc[(school_data_complete["grade"] == "11th"), :]
avg_read_11th = avg_read_11th.groupby(["school_name"]).mean()["reading_score"]

avg_read_12th = school_data_complete.loc[(school_data_complete["grade"] == "12th"), :]
avg_read_12th = avg_read_12th.groupby(["school_name"]).mean()["reading_score"]

read_by_grade_df = pd.DataFrame({"9th": round(avg_read_9th, 2),
                                 "10th": round(avg_read_10th, 2),
                                 "11th": round(avg_read_11th, 2),
                                 "12th": round(avg_read_12th, 2)
                                })

# Rename the index column
read_by_grade_df.index.names = ["School Name"]

read_by_grade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [369]:
bins = [0, 584.99, 630, 645, 680]
labels = ["<$585", "$585-$630", "$630-$645", "$645-$680"]

pd.cut(per_student_budget.iloc[:, 1], bins, labels = labels).head()
per_student_budget

IndexingError: Too many indexers

## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
